In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import LogisticRegression 

from sklearn.neighbors import KNeighborsRegressor


from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, cross_val_score


from sklearn.metrics import r2_score

from matplotlib.colors import ListedColormap, colorConverter, LinearSegmentedColormap

df = pd.read_csv("german.data", sep=" ", header=None,names=['checking','duration','history','purpose','credit_amt','savings','employment','installment_rste','personal_status',
                                                           'other_debtors','resident_since','property','age','other_plans','housing','credits_at_bank','job','liable_maintenance','telephone','foreign_worker','credit_risk'])

# Representing Data Assignment #

## Eploratory Data analysis ##

The first two lines of code above imported the packages you should nee and the data you will be working on. This dataset classifies people described by a set of attributes as good or bad credit risks.**You want to answer the question what makes a creditor good or bad?** The first thing you should do is explore the data. look at what the features are, what type they are and how you might use some or all of them to predict wether they will be a good or bad creditor. you may want to include distrributions or boxplots of some of the features. You can include One-hot-encoding here. Describe your finidings in a few sentences.

TO DO: Finidings

## Baseline Model ##

Now based on your exploration you should have noted that some features are categorical and some are numerical. As we have learned building a model only makes sense if the data is represented in a numerical way. Therfore, to get a baseline create a classification model of your choosing (ie. knn, logistic, SVM, decision tree, etc.) and split the data into training and testing sets using only numeric features. Report the accuracy of your model on the test set.

To Do: accuracy

## One-hot encoding ##

Now that we have a baseline for only the muneric data lets start thinking about the categorical data. Convert the categorical features into a numerical format and train your model using these features. Again report the accuracy  of this model and describe any differences that may occur in the two models.

To Do: accuracy and comparison

## Feature interactions, Transformations, and Selection ##

The next steps will be to possibly reduce the number of features, transform some features, and possibly create new features. Use what we have discussed in class and your exploratory analysis of the data to develop a model that increases your accuracy by 2-5%. It is okay if you are not able to obtain that much of an increase as most of the will be based on your process and analysis. You should try to explain what your finidings are and why you tried something along the way.

To Do: Analysis

## Reflection ##

The last part is to reflect on your work. What worked, what did not. What features seemed super important which ones did not. Was there anything that you thought this will work but did not change the model much.